## Lane detection in a video

In [1]:
import cv2
import numpy as np

def make_coordinates(image, line_parameters):
	slope, intercept = line_parameters
	y1 = image.shape [0]
	y2 = int(y1*(3/5))
	x1 = int((y1 - intercept)/slope)
	x2 = int((y2 - intercept)/slope) 
	return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
	left_lane = []
	right_lane = []
	for line in lines:
		x1, y1, x2, y2 = line.reshape(4)
		parameters = np.polyfit((x1, x2), (y1, y2), 1)
		slope = parameters[0] 
		intercept = parameters[1]
		if slope < 0:
			left_lane.append((slope, intercept))
		else:
			right_lane.append((slope, intercept))
	left_lane_average = np.average(left_lane, axis=0) 
	right_lane_average = np.average(right_lane, axis = 0)
	left_line = make_coordinates(image, left_lane_average) 
	right_line = make_coordinates(image, right_lane_average)
	return np.array([left_line, right_line])

def canny(image):
	gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
	blur= cv2.GaussianBlur(gray, (5, 5),0)
	canny = cv2.Canny (blur, 50, 150)
	return canny

def draw_lines(image, lines):
	line_image = np.zeros_like(image)
	if lines is not None:
		for x1, y1, x2, y2 in lines:
			cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
	return line_image

def region_of_interest(image): 
	height= image.shape[0]
	triangle = np.array([[(200, height ), (1100, height), (550, 250)]])
	mask = np.zeros_like(image) 
	cv2.fillPoly(mask, triangle, 255)
	masked_image = cv2.bitwise_and(image, mask) 
	return masked_image

video = cv2.VideoCapture('test_video.mp4')

while(video.isOpened()):
    _, frame = video.read()
    
    canny_image = canny(frame)

    roi_image= region_of_interest(canny_image)

    lines = cv2.HoughLinesP(roi_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)

    averaged_lines = average_slope_intercept(frame, lines)

    line_image = draw_lines(frame, averaged_lines)

    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)

    cv2.imshow("result", combo_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video.release()
cv2.destroyAllWindows()